# Лабараторная работа 1. MLP.

На библиотеке "Keras" реализовать MLP.

В качестве примера взять простой набор данных из Sklearn:
http://scikit-learn.org/stable/datasets/#sample-generators

MLP сделать из двух слоёв.

In [ ]:
#Подключаем нужнае библиотеки:
import numpy as np

# Библиотеки sklearn, для данных, их подготовки, нормализации
from sklearn import datasets
from sklearn.cross_validation import *
from sklearn.metrics import *
from sklearn.preprocessing import *

# Графики
import matplotlib.pyplot as plt

# Модель нейронной сети
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import *

%matplotlib inline

Подготовка данных:

In [ ]:
# Устанавливаем количество примеров
n_samples = 1500

# Загружаем набор данных из sklearn
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)
x, y = noisy_circles

# Нормализуем данные:
# Преобразует данные в значения от 0 до 1
X = MinMaxScaler().fit_transform(x)

# Преобразует список результатов в бинарную матрицу
# [0, 1] - > [[1. 0.], 
#             [0. 1.]]
Y = np_utils.to_categorical(y, 2)

#Делим загруженные данные на обучающую и тестировочную выборки
train_x, test_x, train_y, test_y = train_test_split(X, Y, train_size=0.8)

In [ ]:
# Визуализация исходного множества:
plt.figure()
plt.title("Original space")
reds = y == 0
blues = y == 1

plt.plot(x[reds, 0], x[reds, 1], "ro")
plt.plot(x[blues, 0], x[blues, 1], "bo")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

plt.show()

Построение модели:

In [ ]:
# Создаём последовательную модель, которая представляет собой линейный стек слоев.
model = Sequential()

# Добавляем входной слой, с 4 нейронами (= число выходных сигналов) и числом входных сигналов (у нас это две координаты точек) 
model.add(Dense(4, input_shape=train_x.shape[1:]))
# Устанавливаем функцию активации - гиперболический тангенс: 'tanh'.
model.add(Activation('tanh'))

#Добавляем ещё один скрытый слой c 2 нейронами.
model.add(Dense(2))
model.add(Activation('tanh'))

# Добавляем выходной слой. Он состоит из 2 нейронов, потому что у нас 2 выходных значения
model.add(Dense(2))
# Применяем Функцию активации 'softmax'
# Эта функция активации помогает трактовать выходной слой, как вероятность событий, так как сумма выходов слоя равняется единице.
model.add(Activation('softmax'))

# Настраиваем процесс обучения с одной из функций оптимизации и функций ошибки. Устанавливаем рассматриваемое значение метрики - точность(=['accuracy'])
# Функция оптимизации: 'SGD' c leaning rate: 0.1 (lr=0.1)
# Функция ошибки: 'mse'
sgd = SGD(lr=0.1, decay=0.0, momentum=0.0)
model.compile(loss='mse', optimizer=sgd, metrics=['accuracy'])

#Параметры ранней остановки и сохранение лучшей модели
early_stopper = EarlyStopping(monitor='val_loss', patience=90)
callback_save_model = ModelCheckpoint("models/Lab1.hdf5", save_best_only=True)

# Обучаем построенную модель на 150 эпохах и batch_size=10 с validation_split=0.1
history = model.fit(train_x, train_y, nb_epoch=150, validation_split = 0.1, batch_size=10, callbacks = [callback_save_model, early_stopper])
# Проверяем модель на тестировочной выборке с verbose=0
# параметр verbose - отвечает за протоколирование процесса. 0 - нет протоколирования, 1 - есть.
score = model.evaluate(test_x, test_y, verbose=0)

#Выводим на экран получившуюся точность нашей модели
print('Test accuracy:', score[1])

# Схема нейрона:
<img src="http://www.5byte.ru/book/1/images/ris11.gif">
F(S) - функция активации.


# Функция активации: Гиперболический тангенс.
<img src="https://hsto.org/files/c71/db2/a75/c71db2a756494e5298ed1d5b5f15cbc9.png">

In [ ]:
# Функция активации f(x), гиперболический тангенс:
import math
def tanhen(x):
    a = []
    for item in x:
        a.append((math.exp(2*item)-1)/(math.exp(2*item)+1))
    return a

x = np.arange(-10., 10., 0.2)
sig = tanhen(x)
plt.plot(x,sig)
plt.title("f(x): tanh", fontsize=25)
plt.show()

# Функция ошибки: MSE (Mean Squared Error) - Среднеквадратическая ошибка

За каждую эпоху, мы считаем ошибку, отняв от идеального ответа, полученный. Далее, возводим в квадрат, после чего полученное число делим на количество эпох.
    
   /* Примечание: 
       Ошибка может считаться каждую эпоху. В частности, keras считает ошибку каждую эпоху, а также каждый batch_size. */

$MSE = \frac{(i_1-a_1)^2+(i_2-a_2)^2+...+(i_n-a_n)^2}{n}$, где: 

i - нужное значение,

a - полученное значение,

n - количество эпох.

# Оптимизационная функция SGD (Stochastic Gradient Descent) - Стохастический градиентный спуск

Эта функция позволяет менять веса нейронов. 
Обратное распространение ошибки не вносит в работу сети оптимизацию. Оно перемещает неверную информацию с конца сети на все веса внутри, чтобы другой алгоритм (в нашем случае SGD) уже смог оптимизировать эти веса так, чтобы они соответствовали нашим данным.

Корректировка коэффициентов сети производится после каждого элемента обучения, где значение градиента аппроксимируются градиентом функции стоимости, вычисленном только на одном элементе обучения.

$Q(w) = \sum_{i=1}^l L(a(x_i, w), y_i)-> min(w)$, где

$L(a,y)$ заданная функция ошибки.

На каждой итерации алгоритма вектор w изменяется в направлении наибольшего убывания функционала Q (то есть в направлении антиградиента):

$w := w-\eta \nabla Q(w)$

In [ ]:
# Визуализация 2-ух кривых ошибок:
plt.figure()
plt.title("Training loss and validation loss")
plt.plot(history.history["loss"], label = 'train')
plt.plot(history.history["val_loss"], label = 'valid')
plt.xlabel("$Epoch$")
plt.ylabel("$Loss$")
plt.legend()
plt.show()